In [4]:
import tensorflow as tf 
import tensorflow.examples.tutorials.mnist.input_data as input_data

#加载mnist数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)     

#输入的数据和标签的占位符
x = tf.placeholder(tf.float32, [None, 784])
y_labels = tf.placeholder(tf.float32, shape=[None, 10])


#初始化权重
def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
#初始化偏差
def bias_var(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#卷积
def convolution(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
#最大池化
def maxPool2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#进行卷积

x_image = tf.reshape(x, [-1, 28, 28, 1])

##一层卷积
Wconvo1 = weight_var([5, 5, 1, 32])
bconvo1 = bias_var([32])

hconvo1 = tf.nn.relu(convolution(x_image, Wconvo1) + bconvo1)
hpool1 = maxPool2x2(hconvo1)
##二层卷积
Wconvo2 = weight_var([5, 5, 32, 64])
bconvo2 = bias_var([64])

hconvo2 = tf.nn.relu(convolution(hpool1, Wconvo2) + bconvo2)
hpool2 = maxPool2x2(hconvo2)

#密集连接层
Wfc1 = weight_var([7 * 7 * 64, 1024])
bfc1 = bias_var([1024])

hpool2flat = tf.reshape(hpool2, [-1, 7*7*64])
hfc1 = tf.nn.relu(tf.matmul(hpool2flat, Wfc1) + bfc1)

#减少过拟合操作
keep_prob = tf.placeholder("float")
hfc1drop = tf.nn.dropout(hfc1, keep_prob)

#output layer
Wfc2 = weight_var([1024, 10])
bfc2 = bias_var([10])

y_conv = tf.nn.softmax(tf.matmul(hfc1drop, Wfc2) + bfc2)

cross_entropy = -tf.reduce_sum(y_labels*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

for i in range(1001):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        trainAccuracy = accuracy.eval(feed_dict={
            x: batch[0], y_labels: batch[1], keep_prob: 1.0
        })
        print("[step %d] , [训练正确率]:%g" % (i, trainAccuracy))
    train_step.run(feed_dict={x: batch[0], y_labels: batch[1], keep_prob: 0.5})

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
[step 0] , [训练正确率]:0.2
[step 100] , [训练正确率]:0.68
[step 200] , [训练正确率]:0.86
[step 300] , [训练正确率]:0.84
[step 400] , [训练正确率]:0.74
[step 500] , [训练正确率]:0.84
[step 600] , [训练正确率]:0.82
[step 700] , [训练正确率]:0.84
[step 800] , [训练正确率]:0.84
[step 900] , [训练正确率]:0.9
[step 1000] , [训练正确率]:0.82
